https://www.cdm.depaul.edu/

In [2]:
import os
from urllib.request import urlopen, urljoin, urlparse, Request
from html.parser import HTMLParser
import collections
from collections import Counter
import re
import threading
from queue import Queue
#Allows to create directory. 

In [3]:
def createProject (directory):
    '''Create folder to store files keep track of each web page.'''
    
    if not os.path.exists(directory):
    #If the path is not exits. 
        os.makedirs(directory)
        #Create directory recursively.

In [4]:
def write_file (linkPath, data):
    '''Write links to outfiles.'''
    
    outfile = open(linkPath, 'w')
    outfile.write(data)
    outfile.close()

In [5]:
def createFiles(projectName, baseURL):
    '''Create files within folders above.'''
    
    waitLinks = projectName + '/waitLink.txt'
    #Store a list of url links waiting to be crawled. Constantly updated. 
    crawledLinks = projectName + '/crawledLink.txt'
    #Store a list of url links were crawled. Constantly updated. 
    
    #After a web page is crawled, the link will move from waitLinks to crawledLinks. 
    if not os.path.isfile(waitLinks):
    #If the file does not exists.
        write_file(waitLinks, baseURL)
        #Create a new text file, using baseURL as initial page (starting point).
        
    if not os.path.isfile(crawledLinks):
        write_file(crawledLinks, '')
        #Initially will be empty, because it hasn't crawled any website yet.

In [6]:
def appendFiles(linkPath, data):
    '''Adding data to existing file.'''
    
    with open (linkPath, 'a') as infile: 
        infile.write(data + '\n')
        #Adding new links to file. 

In [7]:
def deleteFiles_link(linkPath):
    '''Delete content in a file.'''
    
    with open (linkPath, 'w'): pass
    #Open file 'w' will wipe all current content. 

In [8]:
def file_Set(fileName):
    '''Read a file and convert each link to a set() item.'''
    
    ans = set()
    with open (fileName, 'rt') as infile:
    #Open and read file the file as text. 
        for link in infile:
            ans.add(link.replace('\n', ''))
            #Delete the new line character added in def appendFiles().
    return ans

In [9]:
def linkset_Files(links, file):
    '''Iterate through a set, each link will be converted to a new line in a file.'''
    #Links set() is obtained through collector(HTMLparser)
    
    deleteFiles_link(file)
    #Wipe all current content because link set() has updated url links. 
    for link in links:
        appendFiles(file, link)

In [10]:
class Collector(HTMLParser):
    """Collects hyperlink URLs into a list"""

    ignore_tags = ['script', 'noscript', 'input', 'meta', 'title', 'style', 'form']
    tag = ''
    def __init__(self, base_url):
        """ initializes parser, the url, and a list """
        HTMLParser.__init__(self)
        self.base_url = base_url
        #self.page_url = page_url
        self.links = set()
        self.trash = []
        self.data = []

    def handle_starttag(self, tag, attrs):
        """ collects hyperlink URLs in their absolute format """      
        self.tag = tag
        #Assign tag to an instance variable.
        if self.tag == 'a':
        #<a> tag defines a hyperlink
        
            for (attribute,value) in attrs:
            #Iterate over each piece of HTML. 
                if attribute == 'href':
                #Specifies a URL.  
                
                    absolute = urljoin(self.base_url, value.strip().lower())
                    #Using strip() because ' https://depaulmagazine.com/2022/12/08/professional-partners/' has leading whitespace 
                    #Assign absolute variable: Construct absolute URL, join base URL and value.
                    
                    if absolute[:4] == 'http':
                        self.links.add(absolute)
                        if absolute.find('course-evaluation') == -1: 
                            self.trash.append(absolute)
                    else: self.trash.append(absolute)
                            
                    
    def handle_data(self, data):
        if self.tag not in self.ignore_tags:
            for word in data.strip().split():
                gen_word = word.lower()
                self.data.append(gen_word)

    def getLinks(self):
        """ returns hyperlinks URLs in their absolute format """
        return self.links

    def getTrash(self):
        """ returns hyperlinks URLs in their absolute format """
        return self.trash
    
    def getData(self):
        """ returns the data (accumulated in the instance variable) """
        return self.data

In [55]:
class masterCrawl: 
    
    projectName = ''
    baseURL = ''
    domainName = '' 
    #Making sure crawling within CDM website
    
    waitFile = ''
    #waitFile to store wait set()
    crawledFile = ''
    #crawledFile to store crawled set()
    
    wait = set()
    crawled = set()
    
       
    def __init__ (self, projectName, baseURL, domainName):
        masterCrawl.projectName = projectName
        masterCrawl.baseURL = baseURL
        masterCrawl.domainName = domainName
        masterCrawl.waitFile = masterCrawl.projectName + '/waitLink.txt'
        masterCrawl.crawledFile = masterCrawl.projectName + '/crawledLink.txt'
        self.boot()
        self.crawl('Crawl 1', masterCrawl.baseURL)
    
    @staticmethod
    def boot():
        '''Asking first crawl to create directory and files.'''
        createProject (masterCrawl.projectName)
        createFiles(masterCrawl.projectName, masterCrawl.baseURL)
        masterCrawl.wait = file_Set(masterCrawl.waitFile)
        masterCrawl.crawled = file_Set(masterCrawl.crawledFile)
        
    @staticmethod
    def crawl(crawl_num, pageURL):
        ''' Recursive web crawler that calls analyze() on every visited web page.'''
        if pageURL not in masterCrawl.crawled:
            print(crawl_num + ':' + pageURL)
            print ('Wait  ' + str(len(masterCrawl.wait)) + '  | Crawled  ' + str(len(masterCrawl.crawled)))
            masterCrawl.addWaitlinks(masterCrawl.compileLinks(pageURL))
            #Retrieve a list of respective hyperlink in pageURL. Then feed to waitFile.
            masterCrawl.wait.remove(pageURL)
            #Remove from wait file
            masterCrawl.crawled.add(pageURL)
            #Adding to crawled file
            masterCrawl.updateFiles()
            
    @staticmethod
    def compileLinks(pageURL):
        '''Connect to the webpage, and convert HTML to stirng format. And returns all URLs within page.'''
      
        page_url =  'https://www.cdm.depaul.edu/'
        user_agent = "Mozilla/5.0 (Windows NT 6.1; Win64; x64)"
        request = Request(url)
        request.add_header("User-Agent",user_agent)
        content = urlopen(request).read().decode()
        collector = Collector(url)
        collector.feed(content)
        return collector.getLinks()
        
    
    @staticmethod
    def addWaitlinks(links):
        '''Takes a set of link and add to existed waiting list.'''
        for url in links:
            if url in masterCrawl.wait:
                continue
            if url in masterCrawl.crawled:
                continue 
            if masterCrawl.domainName not in url:
                continue
            masterCrawl.wait.add(url)
            #Make sure crawl scope is within CDM 
            
    @staticmethod
    def updateFiles():
        linkset_Files(masterCrawl.wait, masterCrawl.waitFile)
        linkset_Files(masterCrawl.crawled, masterCrawl.crawledFile)     

In [12]:
def sub_domainName(url):
    '''Get the entire domain name.'''
    try:
        return urlparse(url).netloc
    except: return ''

In [13]:
def get_domainName(url):
    '''Get the main domain name. Which is the last two.'''
    try:
        result = sub_domainName(url).split('.')
        return result [-3] + '.' + result [-2] + '.' + result[-1]
    except: return ''        

In [54]:
projectName = 'SurfCDM'
MainURL = 'http://www..cdm.depaul.edu/'
domainName = get_domainName(MainURL)
waitFile = projectName + '/waitLink.txt'
crawledFile = projectName + '/crawledLink.txt'
crawl_num = 8

queue = Queue()
#masterCrawl(projectName, MainURL, domainName)
masterCrawl.addWaitlinks(masterCrawl.compileLinks(MainURL))

{'https://my.cdm.depaul.edu/v2',
 'https://www.cdm.depaul.edu/#ctl00_pagebody',
 'https://www.cdm.depaul.edu/about/pages/about.aspx',
 'https://www.cdm.depaul.edu/about/pages/alert.aspx',
 'https://www.cdm.depaul.edu/about/pages/alumni/alumni.aspx',
 'https://www.cdm.depaul.edu/about/pages/contactus.aspx',
 'https://www.cdm.depaul.edu/about/pages/diversity-equity-inclusion.aspx',
 'https://www.cdm.depaul.edu/about/pages/jobs.aspx',
 'https://www.cdm.depaul.edu/about/pages/partnerships-and-outreach.aspx',
 'https://www.cdm.depaul.edu/about/pages/rsvp.aspx',
 'https://www.cdm.depaul.edu/about/pages/rsvp.aspx?evid=10273',
 'https://www.cdm.depaul.edu/about/pages/rsvp.aspx?evid=10315',
 'https://www.cdm.depaul.edu/about/pages/rsvp.aspx?evid=10345',
 'https://www.cdm.depaul.edu/about/pages/rsvp.aspx?evid=10346',
 'https://www.cdm.depaul.edu/about/pages/rsvp.aspx?evid=10356',
 'https://www.cdm.depaul.edu/about/pages/rsvp.aspx?evid=10360',
 'https://www.cdm.depaul.edu/about/pages/rsvp.aspx?ev

In [55]:
def crawlWork():
    wait_link = file_Set(waitFile)
    if len(waitFile) > 0:
        print(str(len(waitFile)) + ' waiting links')
        createJobs()

In [68]:
def createJobs():
    for link in file_Set(waitFile):
        queue.put(link)
    queue.join()
    crawlWork()

In [66]:
def startCrawl():
    for times in range(crawl_num):
    #Create 8 crawl engines. 
        crawl = threading.Thread(target = work)
        crawl.start()

In [69]:
def work():
    while True:
        url = queue.get()
        masterCrawl.crawl(threading.current_thread().name, url)
        queue.task_done()